In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import scipy.stats
import phi_divergence as phi
from time import process_time


In [49]:
def toymodel(Z_arr):
    x = cp.Variable(2, nonneg = True)
    constraints = [Z_arr @ x <= 1, x[0] <= x[1]-1, x<=10]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value, prob.value)

def toymodel_true(beta):
    x = cp.Variable(2, nonneg = True)
    constraints = [x[0] <= x[1]-1, x<=10, (1-2*beta)*x[1] + 1 >= 0]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value, prob.value)

def lowbound(x,p,r, par, phi_div):
    q = cp.Variable(2, nonneg = True)
    constraints = [cp.sum(q) == 1]
    constraints = phi_div(p,q,r,par,constraints)
    obj = cp.Minimize(q[0])
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(prob.value)

In [61]:
par = 1
phi_div = phi.mod_chi2_cut
n = 1000
phi_dot = 1
r = phi_dot/(2*n)*scipy.stats.chi2.ppf(0.95, 1)

In [23]:
np.random.seed(1)
data = np.random.uniform(-1,1,size = (n,2))
Z_arr = np.array([data[0]])
beta = 0.95
delta = 1/n

In [24]:
lb = -np.inf                      
while beta > lb:
    [x, obj] = toymodel(Z_arr)
    constr = data.dot(x)
    p = np.array([len(constr[constr<=1])/n,len(constr[constr>1])/n])
    lb = lowbound(x,p,r, par, phi_div)
    gap = np.ceil(n*(beta-lb)).astype(int)
    if gap > 0:
        vio_sort = np.sort(constr[constr>1])    #the violations are ranked here
        vio_value = vio_sort[gap-1]
        ind = np.where(constr == vio_value)[0][0]   # the n*(beta-lb)-th scenario is added
        Z_arr = np.append(Z_arr, np.array([data[ind]]),axis = 0)
print('true prob', 1/2+1/(2*x[1]))
print(obj)
print(lb)
print(len(Z_arr))

true prob 0.9666490791365197
1.1429379049679955
0.9580573787418731
3


In [68]:
np.random.seed(1)
data = np.random.uniform(-1,1,size = (n,2))
Z_arr = np.array([data[0]])
beta = 0.85
delta = 1/n

In [56]:
lb = -np.inf
while beta > lb:
    [x, obj] = toymodel(Z_arr)
    constr = data.dot(x)
    p = np.array([len(constr[constr<=1])/n,len(constr[constr>1])/n])
    lb = lowbound(x,p,r, par, phi_div)
    if len(constr[constr>1])!= 0:
        vio_min = np.min(constr[constr>1])        # the least violated scenario is added
        ind = np.where(constr == vio_min)[0][0]
        Z_arr = np.append(Z_arr, np.array([data[ind]]),axis = 0)
print(obj)
print(lb)
print('true prob', 1/2+1/(2*x[1]))
print(len(Z_arr))

1.1565613150048386
0.952515434630034
true prob 0.9637011677072378
202


In [69]:
[x_true, obj_true] = toymodel_true(beta)
constr = data.dot(x_true)
p = np.array([len(constr[constr<=1])/n,len(constr[constr>1])/n])
lb = lowbound(x_true,p,r, par, phi_div)
print(p)
print(lb)
print(obj_true)

[0.834 0.166]
0.8176931077100779
1.8571428571428572


In [52]:
constr

array([ 0.50932224, -0.74411138, -1.19577557, ..., -1.38762293,
       -0.99387235,  0.4002081 ])